In [ ]:
%%html
<div>
<font style="font-size: xx-large; font-weight: bold">Simulating KEM with DH: how and why</font>
</div>
<div>By <a href="https://rlc.vlinder.ca" target="_blank">Ronald Landheer-Cieslak</a></div>
<div style="text-align: right">
<div><a target="_blank" href="index.pdf">Download PDF version</a></div>
<div><a target="_blank" href="https://github.com/blytkerchan/ecdh-kem.git">GitHub</a></div>
<div><a target="_blank" href="https://applied-paranoia.com">Applied Paranoia</a></div>
</div>

***ROUGHT DRAFT***

# Introduction

With the advent of quantum computers, [quantum supremacy having been shown](https://www.nature.com/articles/s41586-019-1666-5)<cite data-cite="arute2019quantum"></cite> and [quantum practicality](https://m-cacm.acm.org/magazines/2023/5/272276-disentangling-hype-from-practicality-on-realistically-achieving-quantum-advantage/fulltext)<cite data-cite="hoefler2023disentangling"></cite> only being a decade or so away, NIST is in its [fourth round of looking for post-quantum cryptography algorithms](https://csrc.nist.gov/projects/post-quantum-cryptography)<cite data-cite="nistpqc"></cite>.

Novel algorithms like [SWOOSH](https://cryptosith.org/papers/swoosh-20230223.pdf)<cite data-cite="gajland2023swoosh"></cite> notwithstanding, no Diffie-Hellman-type algorithms have so far been chosen as [standardization candidates](https://csrc.nist.gov/Projects/post-quantum-cryptography/selected-algorithms-2022)<cite data-cite="nistselected"></cite> but three out of the four announced algorithms are Key Encapsulation Mechanisms rather than Key Exchange Mechanisms (KXM?).

With that in mind, I decided to show how you can simply turn a Diffie-Hellman-type key exchange mechanism into the functional equivalent of a KEM, implementing the same generic API as a KEM, but using only ECHD primitives. The choice of ECDH rather than classical DH is arbitrary. The intent is to use an approach like this to determine whether proposed changes to novel or existing protocols, replacing a DH-based key establishment approach with one based on KEM, is feasible.

# What is a KEM?

A KEM is used, as the name implies, to encapsulate a (symmetric) key in order to securely send it to someone else. As it's an asymmetric cryptography algorithm, you need the other person's public key to do this but, unlike key exchange algorithms, the other person does not need your public key to receive the symmetric key and, unlike digital signature algorithms and key exchange algorithms, the recipient cannot authenticate the received key with only this mechanism.

This is actually one of the oldest applications of RSA besides digital signatures: because RSA allows things encrypted with a public key to be  decrypted with the corresponding private key as well as allowing  something encrypted with a private key to be decrypted with the corresponding public key, it can be used both for key encapsulation and signatures.

## KEM API

The KEM API consists of three functions:

- keypair generation: $G \rightarrow \langle sk,pk\rangle$ generates a key pair $\langle sk,pk\rangle$ where $sk$ is the private key and $pk$ is the public key.
- encrypting: $E \rightarrow pk \rightarrow \langle s,ct \rangle$ takes the public key $pk$ and generates a random shared secret $s$ and the corresponding ciphertext $ct$. Due to the way the API is specified (which is essentially the smallest common denominator for these types of algorithms) the user has no control over the value of the shared secret. We will exploit this fact in our simulation.
- decrypting: $D \rightarrow sk \rightarrow ct \rightarrow s$ takes the private key $sk$ and the ciphertext $ct$ and produces the shared secret, which is identical to the one generated by $E$.

In the proof of concept below, $G$ is called `kem_gen`, $E$ is called `kem_enc` and $D$ is called `kem_dec`.

## KEM use case

The typical use case for a key encapsulation is the same as for a key exchange: Alice wants to send a message to Bob and doesn't want Eve to be able to eavesdrop. To do this:

1. Bob generates a key pair (private key and public key) and securely provisions his public key to Alice (i.e. she can authenticate it). More formally, Bob performs $\langle sk, pk \rangle = G$, then signs $pk$ and sends it to Alice.
2. Alice uses Bob's public key to generate a shared secret and a ciphertext. She then encrypts her message with that derived key using an AEAD. She then signs the key ciphertext and the encrypted message and sends it all to Bob. More formally, she sends $\langle m', ct, signature\rangle$ where $m'$ is the encrypted message and $ct$ is the ciphertext of the random key.
3. Bob receives the message, verifies the signature, decrypts the ciphertext using his private key and thus obtains  the same shared secret as Alice, and uses that the decrypt the message. 

Eve will only have seen the public key and ciphertext, and thus has no feasible way to recover the shared secret or the message.

Note that the signatures above require a separate digital signature algorithm, not provided by KEM itself.

# How is a KEM different from DH?

The use case between a KEM and a DH is very similar: on both cases, Alice and Bob want to establish a symmetric key to use in communications. In the case of a DH, this happens by Alice and Bob exchanging their respective public keys, and each using their respective private keys and each others' public keys.

While that use case is similar, there are significant differences: KEM does not provide mutual authentication where DH does, which means KEM requires the use of a separate digital signature algorithm *for every message*: DH also requires the use of a signature algorithm, but only during public key exchanges while KEM requires it every time a new shared key is generated.

Similarly to KEM, DH does not allow its user any influence over the contents of the symmetric key. Also, because with the use of the same keys the resulting shared secret is always the same, it is recommended to always include a salting step (generally in the form of a publicly-communicated salt and an HKDF function), though that is not generally "spelled out" for KEMs (yet).  

## The DH API

The DH API consists of two functions:

- keypair generation: $G \rightarrow \langle sk, pk \rangle$ generates a keypair $\langle sk, pk \rangle$ where $sk$ is the private key and $ks$ is the public key, and
- shared secret generation: $X \rightarrow sk \rightarrow fpk \rightarrow s$ where $sk$ is the local private key and $fpk$ is the foreign (remote) public key, and $s$ is the generated shared secret.

Because there are only two functions, and the exchange function $X$ is symmetrical (in that $X~sk_a~pk_b = X~sk_b~pk_a$), generation of the shared secret can be done without any on-line communication, provided the foreign public key $fpk$ is known, and if the foreign public key $fpk$ is trusted by the recipient of a message, no further signatures are necessary. This means that if Alice wants to send a message to Bob, and she wants to use a unique symmetric key to encrypt it (say using an AEAD), she only has to send the ciphertext of her message and the salt used to generate the symmetric key ($\langle m, salt\rangle$) to Bob. With KEM, the equivalent would be to generate a a shared secret and corresponding ciphertext, and send both the ciphertext and the salt along with the message to Bob, signing the entire message with a separate private key so Bob can verify the message, sending $\langle m, ct, signature\rangle$.

## The KEM use case using DH

With this in mind, let's take another look at our use case: Alice wants to send a message to Bob without Eve being able to eavesdrop.

1. Alice and Bob each generate key pairs and exchange their public keys. Like in KEM, provisioning the keys is done securely such that the other party can authenticate the public key but unlike KEM, public keys are exchanged in both directions.
2. Alice uses her own private key and Bob's public key to generate a shared secret, and uses that shared secret and a salt to create a derived symmetric key. She prepares her message and encrypts it using an AEAD, and sends the encrypted message and the salt to Bob.
3. Bob receives the salt and the encrypted message. He uses his own private key and Alice's public key to generate the shared secret, and uses Alice's salt to create the derived symmetric key. He can then use the derived symmetric key to decrypt and authenticate the message using the AEAD.



# Building a KEM

Regardless of any superiority DH may have over KEM, as mentioned above, there are currently no selected candidates for standardization for post-quantum cryptography in the DH family. We therefore need to look at two things:

1. How can we build a KEM algorithm today, using pre-quantum tools?
2. How can we use KEM in our existing protocols where we were using DH before?

For the first question, two possible answers come to mind:

1. use RSA: RSA supports both digital signatures by encrypting a hash value with the RSA private key (where verification decrypts the value with the RSA public key) and key encapsulation by encrypting the key value with the RSA public key and decrypting it with the corresponding RSA private key.
2. use DH: converting a DH algorithm into a KEM is trivial, as shown below.

For the second question, the answer will depend on the protocol.

## Using RSA as a KEM

The Python code below is an example of implementing the three KEM functions, `kem_gen`, `kem_enc` and `kem_dec` using RSA.

In [ ]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
import os

KEY_SIZE=2048

def kem_gen():
    sk = rsa.generate_private_key(public_exponent=65537, key_size=KEY_SIZE)
    pk = sk.public_key()
    return sk, pk

def kem_enc(pk):
    s = os.urandom(32)
    ct = pk.encrypt(
        s,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return s, ct

def kem_dec(sk, ct):
    return sk.decrypt(
        ct,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

# Bob:
sk, pk = kem_gen()
# send pk to Alice

# Alice:
s_b, ct = kem_enc(pk)
# send ct to Bob

# Bob:
s_a = kem_dec(sk, ct)

s_a == s_b

As shown, the RSA version requires some padding but can easily be used to implement the KEM API. Over-the-wire communications are not represented in this example.

## Building a KEM using DH

To build a KEM using only an ECDH, the approach is a bit different than it is with RSA and with "real" KEMs: while the generation function still returns a private and a public key, the encryption function actually generates an ephemeral ECDH key pair, of which the private key is used to generate the shared secret, and the public key is presented as the "ciphertext" of that shared secret. While that "ciphertext" as no real relation to the shared secret, it can be used to re-generate the same shared secret when using the original, non-ephemeral private key.

That means that, when using a KEM, Bob still generates a keypair $\langle sk_b, pk_b\rangle$ and provides his public key $pk_b$ to Alice, but Alice now generates her own ephemeral keypair $\langle sk_a, pk_a\rangle$ and generates the shared secret $$s = X~sk_a~pk_b$$

She them provides $pk_a$ to Bob who generates the shared secret $$s = X~sk_b~pk_a$$

With $ct = pk_a$, this is exactly equivalent to $$E \rightarrow pk_b \rightarrow s,ct\\E = s, ct\\s = X~sk_a~pk_b\\sk_a, ct = G$$

or in Python:

In [ ]:
from cryptography.hazmat.primitives.asymmetric import ec

curve = ec.SECP384R1()

def kem_gen():
    sk = ec.generate_private_key(curve)
    pk = sk.public_key()
    return sk, pk

def kem_enc(pk):
    fsk = ec.generate_private_key(curve)
    ct = fsk.public_key()
    s = fsk.exchange(ec.ECDH(), pk)
    return s, ct

def kem_dec(sk, ct):
    s = sk.exchange(ec.ECDH(), ct)
    return s

# Bob:
sk, pk = kem_gen()
# send pk to Alice

# Alice:
s_b, ct = kem_enc(pk)
# send ct to Bob

# Bob:
s_a = kem_dec(sk, ct)

s_a == s_b

Using DH shared secrets directly is generally frowned upon (with good reason) whereas using KEM secrets directly is not, so a more secure implementation in Python would look like this:

In [ ]:
import os
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

curve = ec.SECP384R1()

def kem_gen():
    sk = ec.generate_private_key(curve)
    pk = sk.public_key()
    return sk, pk

def kem_enc(pk):
    fsk = ec.generate_private_key(curve)
    salt = os.urandom(32) # 256 bits may be a bit overkill
    ct = fsk.public_key(), salt
    ss = fsk.exchange(ec.ECDH(), pk)
    hkdf = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        info=None
    )
    s = hkdf.derive(ss)
    return s, ct

def kem_dec(sk, ct):
    pubkey, salt = ct
    ss = sk.exchange(ec.ECDH(), pubkey)
    hkdf = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        info=None
    )
    s = hkdf.derive(ss)
    return s

# Bob:
sk, pk = kem_gen()
# send pk to Alice

# Alice:
s_b, ct = kem_enc(pk)
# send ct to Bob

# Bob:
s_a = kem_dec(sk, ct)

s_a == s_b

This adds an additional HKDF step to the shared secret generation, and adds a salt value to the "ciphertext" sent from Alice to Bob, but does not otherwise change the API.

# Using a KEM in protocols

Most protocols that use an DH-type algorithm today already also use a digital signature algorithm, if only to sign the DH public key exchanged between "Alice" and Bob. For those protocols, the message from Bob to Alice sending Bob's public key does not really change: it needs to be signed by Bob and Alice needs to verify that signature, but they had to do that already. The main difference will be in the order of the handshake: if the handshake had Alice send her public key to Bob before Bob sent his public key to Alice, the protocol may need to become one where Bob ends up generating the shared secret first, and provides the ciphertext for the shared secret to Alice in his response. This is not the case in TLS, but it may be the case in the upcoming DNP3-SAv6.

In DNP3-SAv6, the master station (a.k.a. controlling station in IEC 62351-5) sends its certificates, including its ECDH key, to the outstation (a.k.a. controlled station) in a message called the `AssociationRequest`. The outstation responds with its own certificate, including its ECDH key, in an `AssociationResponse`. Both sides then calculate the Encryption and Authentication Update Keys and prove to each other they have the same keys. Replacing this ECDH-based mechanism with a KEM-based mechanism would have the master send its certificates and KEM public key to the outstation in the `AssociationRequest` message, and have the outstation send its certificates and the $ct$ value in the `AssociationResponse` message. The outstation would then already have the Update Keys (which in DNP3-SAv6 are two 256-bit keys, so an HKDF to lengthen the key may be needed), and the master can decrypt it from the provided $ct$ value.

Alternatively, the `AssociationRequest` message would only contain the master's certificates, the outstation would provide its certificates and a KEM public key in the `AssociationResponse` message, and the master would use the next message in the exchange to provide the ciphertext $ct$ as well as its proof of knowledge of the Update Keys. Upon receipt, the outstation would then decrypt the Update Keys, validate the proof of knowledge, and provide its own to the master.

The cost of generating a KEM key pair, of generating the shared secret and ciphertext, and of decrypting the ciphertext is presently not well-known, so which of these approaches would work best for a protocol like DNP3-SAv6 or IEC 62351-5-derived protocols like IEC 60870-7-5 is hard to tell at this time. It is important to note, though, that it is possible to take either approach, remain compatible with what is currently in the specification in terms of bits-on-the-wire (i.e. the `AssociationRequest` and `AssociationResponse` messages have sufficient flexibility in their format to allow the transfer of a KEM public key and $ct$ value; though arguably the second proposed scheme is a bit harder to work into a DNP technical bulletin than the first one), and retain security in a post-quantum world.